# Import des librairies 
Chargement des librairies

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
df=pd.read_csv('weatherAUS.csv')
df.head()
df['Date']=pd.to_datetime(df['Date'])

Enrichissement avec la quantité de pluie du lendemain

In [2]:
import sys

oldlocation=np.nan
raindayafter=np.nan
dateafter=np.nan
df['qtyraintomorrow']=np.nan

for idx in reversed(df.index):
    row=df.iloc[idx]
    datecurrent=row['Date']
    location=row['Location']  
    if (location==oldlocation)&(datecurrent + pd.DateOffset(days=1)==dateafter):
        df.loc[idx,'qtyraintomorrow']=raindayafter
    else:
        sys.stdout.write("x")
    raindayafter=row['Rainfall']
    oldlocation=location
    dateafter=datecurrent
    


xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

In [3]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,qtyraintomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No,0.0
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No,0.0
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No,0.0
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No,1.0
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No,0.2


# Enrichissement par calcul des variations
Les données sont enrichies par calcul de la variation d'u jour sur l'aure

In [4]:
import sys

oldlocation=np.nan
raindayafter=np.nan
datebefore=np.nan
df['varpressure']=np.nan
df['vartemp']=np.nan
df['varwind']=np.nan
df['varhumidity']=np.nan


for idx in df.index:
    row=df.iloc[idx]
    datecurrent=row['Date']
    pressurecurrent=row['Pressure3pm']
    tempcurrent=row['Temp3pm']
    windcurrent=row['WindGustSpeed']
    humiditycurrent=row['Humidity3pm']
    location=row['Location']  
    if (location==oldlocation)&(datecurrent + pd.DateOffset(days=-1)==datebefore):
        df.loc[idx,'varpressure']=pressurecurrent-pressurebefore
        df.loc[idx,'vartemp']=tempcurrent-tempbefore
        df.loc[idx,'varwind']=windcurrent-windbefore
        df.loc[idx,'varhumidity']=humiditycurrent-humiditybefore
    else:
        sys.stdout.write("x")
    pressurebefore=pressurecurrent
    tempbefore=tempcurrent
    windbefore=windcurrent
    humiditybefore=humiditycurrent
    oldlocation=location
    datebefore=datecurrent
    

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

Sauvegarde

In [5]:
df.to_csv('enricheredaus_sanscluster.csv')

## Interpoler des valeurs manquantes en vue d'une prévision 
D'abord nous construisons un inputer qui peut baser sur une partie des données suivantes:
'Humidity3pm', 'Sunshine', 'varhumidity', 'Pressure9am', 'Rainfall', 'WindGustSpeed','Pressure3pm'

Puis, nous entrainons un knn inputer afin de restituer les valeurs manquantes de potentiel jeu de donnée (qui n'existent pas encore) 

In [19]:
import numpy as np
from sklearn.impute import KNNImputer

columns_to_preserve=['Humidity3pm', 'Sunshine', 'varhumidity', 'Pressure9am', 'Rainfall', 'WindGustSpeed','Pressure3pm']
basedata=df[columns_to_preserve]

imputer = KNNImputer(n_neighbors=3)
imputer.fit(basedata)




   Humidity3pm  Sunshine  varhumidity  Pressure9am  Rainfall  WindGustSpeed  \
0         22.0       NaN          NaN       1007.7       0.6           44.0   
1         25.0       NaN          3.0       1010.6       0.0           44.0   
2         30.0       NaN          5.0       1007.6       0.0           46.0   
3         16.0       NaN        -14.0       1017.6       0.0           24.0   
4         33.0       NaN         17.0       1010.8       1.0           41.0   

   Pressure3pm  
0       1007.1  
1       1007.8  
2       1008.7  
3       1012.8  
4       1006.0  


,Humidity3pm,Sunshine,varhumidity,Pressure9am,Rainfall,WindGustSpeed,Pressure3pm
0,22.0,10.666667,5.0,1010.6,0.2,44.0,1006.0


Chargement du modèle simplifié (Cf. modele-simple.jpnb

In [67]:
import joblib
model = joblib.load("simplified_model.sav")



Ici le modèle est utilisé pour effectué des prédiction sur un échantillon aléatoire. Nous le voyons a l'oeuvre sue le jeu d'entrainement. C'est ce modèle qui sera utilisé pour effectuer des prédiction à aprtir d'un relevé quelquonque sur streamlit.

In [71]:
sample={'Humidity3pm':[22], 'Sunshine':[np.nan], 'varhumidity':[5], 'Pressure9am':[1010.6], 'Rainfall':[0.2], 'WindGustSpeed':[44],'Pressure3pm':[1006]}

samplefromdb=df.iloc[17][columns_to_preserve].array.reshape(1,-1)
sample=pd.DataFrame(columns=columns_to_preserve,data=samplefromdb)
inputtedvalarray=imputer.transform(pd.DataFrame.from_dict(sample))
inputtedval=pd.DataFrame(columns=columns_to_preserve,data=predictedval)
inputtedval

for i in range(50): 
    samplefromdb=df.iloc[i][columns_to_preserve].array.reshape(1,-1)
    real=df.iloc[i]["RainTomorrow"]
    sample=pd.DataFrame(columns=columns_to_preserve,data=samplefromdb)
    
    inputtedvalarray=imputer.transform(pd.DataFrame.from_dict(sample))
    inputtedval=pd.DataFrame(columns=columns_to_preserve,data=inputtedvalarray)
    #print(inputtedval)
    result=model.predict(inputtedval)>.5
    print("Prédiction ",result,", en réalité: ",real)


Prédiction  [False] , en réalité:  No
Prédiction  [False] , en réalité:  No
Prédiction  [False] , en réalité:  No
Prédiction  [False] , en réalité:  No
Prédiction  [False] , en réalité:  No
Prédiction  [False] , en réalité:  No
Prédiction  [False] , en réalité:  No
Prédiction  [False] , en réalité:  No
Prédiction  [False] , en réalité:  Yes
Prédiction  [False] , en réalité:  No
Prédiction  [False] , en réalité:  Yes
Prédiction  [ True] , en réalité:  Yes
Prédiction  [ True] , en réalité:  Yes
Prédiction  [False] , en réalité:  No
Prédiction  [False] , en réalité:  nan
Prédiction  [False] , en réalité:  No
Prédiction  [False] , en réalité:  Yes
Prédiction  [ True] , en réalité:  Yes
Prédiction  [False] , en réalité:  No
Prédiction  [False] , en réalité:  No
Prédiction  [False] , en réalité:  No
Prédiction  [False] , en réalité:  No
Prédiction  [False] , en réalité:  No
Prédiction  [False] , en réalité:  No
Prédiction  [False] , en réalité:  No
Prédiction  [False] , en réalité:  No
Prédi